In [24]:
import pandas as pd
import os
import creating_baskets as cb

In [25]:
#Creating Baskets
market_baskets = cb.CreateBaskets()

In [27]:
#for key, value in market_baskets:
 #   print(key, value)
for item in market_baskets.items():
    print(item[1])

['mens1000free', 'mens100free', 'mens200free', 'mens500free']
['mens1000free', 'mens100free', 'mens200free', 'mens500free']
['mens1000free', 'mens200free', 'mens400IM', 'mens500free']
['mens1000free', 'mens400IM', 'mens500free', 'mens50Free']
['mens1000free', 'mens200free', 'mens400IM', 'mens500free']
['mens1000free', 'mens200fly', 'mens400IM', 'mens500free']
['mens1000free', 'mens200im', 'mens500free']
['mens1000free', 'mens200fly', 'mens200free', 'mens500free']
['mens1000free', 'mens500free']
['mens1000free', 'mens200free', 'mens400IM', 'mens500free']
['mens1000free', 'mens200im', 'mens500free']
['mens1000free', 'mens400IM', 'mens500free']
['mens1000free', 'mens200free', 'mens200im', 'mens500free']
['mens1000free', 'mens200fly', 'mens200free', 'mens500free']
['mens1000free', 'mens200free', 'mens400IM', 'mens500free']
['mens1000free', 'mens100free', 'mens200free', 'mens500free']
['mens1000free', 'mens200fly', 'mens200free', 'mens500free']
['mens1000free', 'mens100free', 'mens200free',